# Introducción a Pandas: Series y DataFrames
**Curso:** Fundamentos de Programación y Analítica de Datos con Python  
Duración estimada: 4 horas

## Objetivos específicos
- Cargar datasets en memoria usando `pandas` desde fuentes comunes (CSV, Excel) y describir su estructura con métodos de inspección (`.head()`, `.info()`, `.describe()`).
- Distinguir entre **Series** y **DataFrames**, creando ambos tipos desde objetos de Python y comprendiendo su indexación básica.
- Seleccionar y filtrar datos con `loc` e `iloc` sobre DataFrames de forma segura y reproducible.
- Aplicar operaciones simples de transformación y agregación inicial (`assign`, operaciones aritméticas columnares, `groupby` + `agg`) en DataFrames.

## Prerrequisitos
Conocimientos básicos de Python (tipos primitivos, listas/diccionarios, funciones) y nociones elementales de NumPy (arrays y operaciones vectorizadas). No se asume experiencia previa con Pandas.


---


## Tema 1 — Series

### Definición
Una **Serie** de Pandas es una estructura unidimensional etiquetada que puede almacenar cualquier tipo de dato (numérico, cadena, booleano, objetos), compuesta por un **array de valores** y un **índice** que etiqueta cada elemento. A diferencia de una lista, la Serie mantiene metadatos (índices) y ofrece operaciones vectorizadas y de alineación por etiquetas.

### Importancia en programación y analítica de datos
- Permite trabajar con vectores etiquetados, facilitando el acceso semántico a los elementos por nombre y no solo por posición.
- Alinea automáticamente los datos por el índice durante operaciones entre Series, reduciendo errores de mezcla/orden.
- Es la pieza base para construir columnas de un DataFrame y realizar transformaciones limpias y reproducibles.


### Buenas prácticas profesionales y errores comunes
- **Buena práctica:** documentar el **significado del índice** y mantenerlo único cuando represente identificadores.  
- **Buena práctica:** preferir operaciones vectorizadas sobre bucles explícitos por rendimiento y claridad.  
- **Error común:** asumir que el índice es siempre 0..n-1; validar el índice con `s.index` y considerar `s.reset_index(drop=True)` cuando se requiera.


In [ ]:

#TODO: Ejemplo básico de Series
import pandas as pd
import numpy as np

#* Serie con índice explícito
poblacion = pd.Series([10_000_000, 50_000_000, 5_150_000], index=['Argentina', 'Colombia', 'Chile'])
print("Serie creada para la población: \n", poblacion)
print("-" * 40, "\n")

#* Realizar acceso por etiquueta y por máscara booleana
print("La población de Colombia es: ", poblacion['Colombia'])
print("Países con población mayor a 8 millones:\n", poblacion[poblacion > 8_000_000], "\n")
print("-" * 40, "\n")

#* Operaciones Vectorizadas
crecimiento = pd.Series([0.01, 0.05, 0.02], index=['Argentina', 'Colombia', 'Chile'])
print("Crecimiento anual relativo: \n", crecimiento)

poblacion_proyectada = poblacion*( 1 + crecimiento)
print("\nPoblación proyectada para el próximo año: \n", poblacion_proyectada, "\n")

#* Observación sobre los indices: Alineación por etiquetas
crecimiento_parcial = pd.Series([0.03, 0.04], index=['Argentina', 'Chile'])
print("Crecimiento parcial: \n", crecimiento_parcial)
print("\nPoblación proyectada con crecimiento parcial: \n", poblacion*(1 + crecimiento_parcial), "\n")

#* -- Julian: Forzar null explícito
crecimiento_null = crecimiento_parcial.reindex(poblacion.index, fill_value=None)
resultado_null = poblacion * (1 + crecimiento_null)
print("-- Crecimiento con None explícito: \n", resultado_null)

#* -- Rafael: Valor anterior sin multiplicar
crecimiento_valor_anterior = crecimiento_parcial.reindex(poblacion.index, fill_value=0)
resultado_valor_anterior = poblacion * (1 + crecimiento_valor_anterior)
print("-- Crecimiento con valor anterior: \n", resultado_valor_anterior)

#* -- Rafael: Valor anterior multiplicando ( mul )
resultado_multiplicando = poblacion.mul(1 + crecimiento_valor_anterior, fill_value=1)
print("-- Crecimiento con valor anterior usando mul: \n", resultado_multiplicando)

Serie creada para la población: 
 Argentina    10000000
Colombia     50000000
Chile         5150000
dtype: int64
---------------------------------------- 

La población de Colombia es:  50000000
Países con población mayor a 8 millones:
 Argentina    10000000
Colombia     50000000
dtype: int64 

---------------------------------------- 

Crecimiento anual relativo: 
 Argentina    0.01
Colombia     0.05
Chile        0.02
dtype: float64

Población proyectada para el próximo año: 
 Argentina    10100000.0
Colombia     52500000.0
Chile         5253000.0
dtype: float64 

Crecimiento parcial: 
 Argentina    0.03
Chile        0.04
dtype: float64

Población proyectada con crecimiento parcial: 
 Argentina    10300000.0
Chile         5356000.0
Colombia            NaN
dtype: float64 

-- Crecimiento con None explícito: 
 Argentina    10300000.0
Colombia            NaN
Chile         5356000.0
dtype: float64
-- Crecimiento con valor anterior: 
 Argentina    10300000.0
Colombia     50000000.0
Chile  

In [53]:
# Agregado adicional
poblacion_proyectada = poblacion*(1 + crecimiento_parcial)
print(poblacion_proyectada)
print("-" * 40, "\n")

print("Suma total: ", poblacion_proyectada.sum())
print("Promedio: ", poblacion_proyectada.mean())
print("Máximo: ", poblacion_proyectada.max())
#* Cuando estamos utiliazndo series numéricas, los datos con NaN son ignorados ( se saltan ) en las operaciones

print(f"Conteo de valores válidos: {poblacion_proyectada.count()} de {poblacion_proyectada.size} registros")

Argentina    10300000.0
Chile         5356000.0
Colombia            NaN
dtype: float64
---------------------------------------- 

Suma total:  15656000.0
Promedio:  7828000.0
Máximo:  10300000.0
Conteo de valores válidos: 2 de 3 registros


---


## Tema 2 — DataFrames

### Definición
Un **DataFrame** es una estructura bidimensional tabular con **filas** e **columnas etiquetadas**, donde cada columna es conceptualmente una Serie y todas comparten el mismo índice de filas. Admite tipos heterogéneos por columna, operaciones vectorizadas y funciones de alto nivel para limpieza y transformación.

### Importancia en programación y analítica de datos
- Modelo mental cercano a **SQL** y **Excel**, ideal para datos tabulares del mundo real.
- Facilita la **inspección** (`.head()`, `.info()`, `.shape`, `.dtypes`), la **selección** (`loc`, `iloc`) y la **transformación** (`assign`, operaciones aritméticas, `groupby`, `agg`).  
- Estandariza flujos de **carga → limpieza → transformación → análisis** con reproducibilidad y trazabilidad.


### Buenas prácticas profesionales y errores comunes
- **Buena práctica:** usar nombres de columnas en **snake_case** sin espacios para evitar errores en accesos y al exportar: `total_ventas`, `fecha_compra`.
- **Buena práctica:** controlar tipos con `df.dtypes` y convertir explícitamente con `astype` cuando sea necesario.
- **Error común:** mezclar `df.col` y `df["col"]` indiscriminadamente; preferir `df["col"]` por seguridad cuando el nombre de columna pueda colisionar con atributos.
- **Error común:** olvidar `index=False` al exportar a CSV/Excel cuando el índice no es semántico.


In [19]:

# TODO: Ejemplo para la creación e inspección de un DataFrame
import pandas as pd

data = {
  "producto": ["A", "B", "C", "B", "A"],
  "precio": [10.0, 20.0, 15.0, 25.0, 12.0],
  "cantidad": [1, 2, 1, 3, 2],
  "ciudad": ["Bogota", "Medellín", "Cali", "Bogotá", "Cali"]
}

df = pd.DataFrame(data)
print(df.head())
print("Dimensiones del DataFrame:", df.shape)
print("Tipos de datos:\n", df.dtypes)
print("Estadísticas descriptivas:\n", df.describe())

#! Sanitización/Normalización de Dataframes


  producto  precio  cantidad    ciudad
0        A    10.0         1    Bogota
1        B    20.0         2  Medellín
2        C    15.0         1      Cali
3        B    25.0         3    Bogotá
4        A    12.0         2      Cali
Dimensiones del DataFrame: (5, 4)
Tipos de datos:
 producto     object
precio      float64
cantidad      int64
ciudad       object
dtype: object
Estadísticas descriptivas:
           precio  cantidad
count   5.000000   5.00000
mean   16.400000   1.80000
std     6.107373   0.83666
min    10.000000   1.00000
25%    12.000000   1.00000
50%    15.000000   2.00000
75%    20.000000   2.00000
max    25.000000   3.00000


In [31]:

# TODO: Ejemplo de Selección y filtrado: loc e iloc
# - loc: Selección por etiquetas ( filas/columnas )
# - iloc: Selección por posición entera ( filas/columnas )

print(df.head())

#* Selección por iloc
# Filas 0..2 y columnas 'producto' y 'precio'
print("Selección por iloc (filas 0 a 2, columnas 0 a 1):")
print(df.iloc[0:3, 0:2])

print("Selección por loc (filas 0 a 2, columnas 'producto' y 'precio'):")
print(df.loc[0:2, ['producto', 'precio']])

# Filtrado condicional
filtro = (df["precio"] <= 14) & (df["ciudad"] == "Bogotá")
print("Filtrado condicional (precio <= 14 y ciudad == 'Bogotá'):")
print(df[filtro])


  producto  precio  cantidad    ciudad
0        A    10.0         1    Bogota
1        B    20.0         2  Medellín
2        C    15.0         1      Cali
3        B    25.0         3    Bogotá
4        A    12.0         2      Cali
Selección por iloc (filas 0 a 2, columnas 0 a 1):
  producto  precio
0        A    10.0
1        B    20.0
2        C    15.0
Selección por loc (filas 0 a 2, columnas 'producto' y 'precio'):
  producto  precio
0        A    10.0
1        B    20.0
2        C    15.0
Filtrado condicional (precio <= 14 y ciudad == 'Bogotá'):
Empty DataFrame
Columns: [producto, precio, cantidad, ciudad]
Index: []


In [ ]:

# TODO: Ejemplo de Transformaciones y agregaciones básicas
#* Nueva columna: valor_total y que sea igual al precio por la cantidad.
df= df.assign(valor_total = df["precio"]*df["cantidad"])
print("DataFrame con nueva columna 'valor_total':\n", df)

# Agregación: Ventas Totales por Producto
ventas_por_producto = df.groupby("producto", as_index=False).agg(
  ventas_totales = ("valor_total","sum"),
  precio_promedio = ("precio","mean"),
  qty = ("producto","count")
)
print("Resumen por producto:\n", ventas_por_producto)

DataFrame con nueva columna 'valor_total':
   producto  precio  cantidad    ciudad  valor_total
0        A    10.0         1    Bogota         10.0
1        B    20.0         2  Medellín         40.0
2        C    15.0         1      Cali         15.0
3        B    25.0         3    Bogotá         75.0
4        A    12.0         2      Cali         24.0
Resumen por producto:
   producto  ventas_totales  precio_promedio  qty
0        A            34.0             11.0    2
1        B           115.0             22.5    2
2        C            15.0             15.0    1


---


# Ejercicios integradores

A continuación, se presentan ejercicios que integran los conceptos de **Series** y **DataFrames**. Se recomienda resolverlos de manera individual y luego discutir en grupo las soluciones y decisiones de diseño.


## Ejercicio 1 — Control de inventario básico con Series
**Contexto técnico:** Eres analista de inventarios en una tienda minorista. Necesitas modelar el stock actual por producto y proyectar el stock tras recibir un lote de reposición. La precisión en el manejo de índices es clave para evitar errores de consolidación.

**Datos/entradas:** 
- Serie `stock_actual` con índices `["A", "B", "C"]` y valores `[20, 35, 12]`.
- Serie `reposicion` con índices `["A", "C"]` y valores `[10, 8]`.

**Requerimientos:**
1. Crear ambas Series con los índices indicados.
2. Calcular `stock_proyectado = stock_actual + reposicion` y explicar por qué aparecen `NaN` cuando una etiqueta no existe en ambas Series.
3. Reemplazar los `NaN` por 0 antes de sumar (pista: `reindex` o `fillna`).

**Criterios de aceptación:**
- `stock_proyectado` tiene índices `["A", "B", "C"]` y valores `[30, 35, 20]`.
- No hay valores `NaN` en el resultado final.

**Pistas:**
- Revise la alineación por etiqueta en Series.
- Use `add` con `fill_value=0` o normalice índices con `reindex`.


In [ ]:

#TODO: Solución del Ejercicio 1

## Ejercicio 2 — Resumen de ventas por producto con DataFrame
**Contexto técnico:** Como analista comercial, necesitas un resumen de ventas por producto para presentar a gerencia. La trazabilidad de cálculos y la correcta selección de columnas son fundamentales.

**Datos/entradas:**
Construye un DataFrame con las columnas `producto`, `precio`, `cantidad` y `ciudad` con al menos 8 filas. Incluye al menos 3 productos y 3 ciudades.

**Requerimientos:**
1. Crear la columna `valor_total = precio * cantidad` usando `assign`.
2. Generar un resumen por `producto` con `ventas_totales` (suma de `valor_total`) y `precio_promedio`.
3. Ordenar el resultado por `ventas_totales` descendente.

**Criterios de aceptación:**
- El DataFrame de salida contiene las columnas `producto`, `ventas_totales`, `precio_promedio` y al menos una fila por producto.
- El orden es descendente por `ventas_totales`.

**Pistas:**
- Use `groupby` y `agg` con nombres de columnas alias.
- Revise `sort_values` para ordenar.


In [ ]:

#TODO: Solución del Ejercicio 2

## Ejercicio 3 — Selección y filtrado para control de calidad
**Contexto técnico:** En un control de calidad, se deben identificar transacciones atípicas por ciudad y revisar casos con precio fuera de un rango esperado.

**Datos/entradas:**
Partiendo del DataFrame `df` del ejercicio anterior.

**Requerimientos:**
1. Seleccionar solo las filas de `Bogota` y `Cali` con `precio >= 10` usando filtrado booleano.
2. Con `loc`, obtener las columnas `producto`, `precio` y `ciudad` de las primeras 5 filas filtradas.
3. Con `iloc`, obtener las filas 0..2 y columnas 0..2 del DataFrame original para comparar.

**Criterios de aceptación:**
- El subconjunto filtrado contiene únicamente las ciudades objetivo con `precio >= 10`.
- Las selecciones con `loc`/`iloc` retornan las dimensiones esperadas.

**Pistas:**
- Combine condiciones con `&` y paréntesis.
- Recuerde que `loc` usa etiquetas y `iloc` posiciones enteras.


In [ ]:

#TODO: Solución del Ejercicio 3

## Ejercicio 4 — Exportación de resultados para reporte
**Contexto técnico:** Necesitas entregar a un colega un archivo intercambiable para continuar el análisis. Se solicita un CSV con el resumen por producto.

**Datos/entradas:**
Usa el resultado `resumen` del Ejercicio 2.

**Requerimientos:**
1. Exportar `resumen` como `resumen_por_producto.csv` sin incluir el índice.
2. Verificar el archivo leyendo nuevamente y comprobando número de filas y columnas.

**Criterios de aceptación:**
- El archivo `resumen_por_producto.csv` existe, no contiene índice y conserva las columnas correctas.
- La verificación muestra el mismo número de filas/columnas que el DataFrame original exportado.

**Pistas:**
- Use `to_csv("resumen_por_producto.csv", index=False)` y luego `pd.read_csv(...)`.


In [ ]:

#TODO: Solución del Ejercicio 4